## Game pattern and bye week constraints

In [1]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import pandas as pd
import numpy as np

In [2]:
Team_list={
0:"Atlanta Falcons",1:"Carolina Panthers",2:"Chicago Bears",3:"Detroit Lions",
4:"Green Bay Packers",5:"Minnesota Vikings",6:"New Orleans Saints",7:"New York Giants",
8:"Philadelpia Eagles",9:"Tampa Bay Buccaneers",10:"Washington Football Team",11:"Baltimore Ravens",
12:"Buffalo Bills",13:"Cincinnati Bengals",14:"Cleveland Browns",15:"Houston Texans",
16:"Indianapolis Colts",17:"Jacksonville Jaguars",18:"Miami Dolphins",19:"New England Patroits",
20:"New York Jets",21:"Pittsburgh Steelers",22:"Tennessee Titans",23:"Dalls Cowboys"}

In [5]:
W=set(list(range(0,12))) #set of weeks, total of 12 weeks
T=set(list(range(0,24))) #set of teams all, total of 24 teams
 #set of conference

#read the distance file
E=pd.read_csv("distance.csv",index_col=0)

In [6]:
W

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

In [7]:
T

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23}

In [8]:
E

,Atlanta Falcons,Carolina Panthers,Chicago Bears,Detroit Lions,Green Bay Packers,Minnesota Vikings,New Orleans Saints,New York Giants,Philadelpia Eagles,Tampa Bay Buccaneers,...,Cleveland Browns,Houston Texans,Indianapolis Colts,Jacksonville Jaguars,Miami Dolphins,New England Patroits,New York Jets,Pittsburgh Steelers,Tennessee Titans,Dallas Cowboys
Atlanta Falcons,0,245,717,722,922,1113,470,864,777,456,...,710,793,534,346,650,1063,864,685,250,793
Carolina Panthers,245,0,756,629,961,1163,715,625,537,580,...,515,1038,585,383,717,824,625,447,409,1026
Chicago Bears,717,756,0,284,207,409,925,787,758,1173,...,344,1082,184,1062,1367,982,787,460,472,967
Detroit Lions,722,629,284,0,489,691,1067,613,583,1178,...,170,1302,287,1010,1385,706,613,286,534,1187
Green Bay Packers,922,961,207,489,0,279,1130,993,964,1382,...,550,1287,394,1272,1578,1189,995,668,683,1173
Minnesota Vikings,1113,1163,409,691,279,0,1199,1195,1165,1580,...,752,1176,591,1435,1774,1389,1195,868,879,940
New Orleans Saints,470,715,925,1067,1130,1199,0,1311,1124,657,...,1051,348,819,546,851,1510,1311,1090,532,505
New York Giants,864,625,787,613,993,1195,1311,0,96,1131,...,456,1629,728,934,1268,204,0,364,886,1549
Philadelpia Eagles,777,537,758,583,964,1165,1124,96,0,1042,...,431,1547,643,846,1179,295,96,305,804,1467
Tampa Bay Buccaneers,456,580,1173,1178,1382,1580,657,1131,1042,0,...,1094,981,990,199,267,1328,1129,1026,705,1103


In [9]:
# Create an empty model
m1= gp.Model(name="NFL1")

Using license file C:\Users\Q_Ali\gurobi.lic
Academic license - for non-commercial use only


#### ADD DECISION VARIABLES 

In [55]:
home=len(E.columns)
away=len(E.columns)
weeks=13 #Now we extend season period to 13 weeks
W=list(range(0,12))
byeweek = W[8]
#if team h plays at home against team j at week k
x = m1.addVars(home, away, weeks, vtype = GRB.BINARY, name = "x")

In [11]:
#if team h plays at home in week k and k+1
y = m1.addVars(home, weeks, vtype = GRB.BINARY, name = "y")

In [12]:
#if team h plays at away in week k and k+1
z = m1.addVars(home, weeks, vtype = GRB.BINARY, name = "z")

#### Add Objective function

In [13]:
# Objective is to minimize the total distance of all teams
#obj = sum(sum(x[w,h,a]*2*E.iloc[h,a] for h in range(home) for a in range(away)) for w in range(weeks))
obj = sum(sum(x[h,a,w]*2*E.iloc[h,a] for w in range(weeks)) for h in range(home) for a in range(away))

#### ADD CONSTRAINTS HERE

In [15]:
#The season was limited to 12 weeks.
con1 = m1.addConstrs(sum(x[h,a,w] + x[a,h,w] for w in range(weeks) for h in range(home)) == 12 
                     for a in range(away))

In [16]:
#each team would play once per week
con2 = m1.addConstrs(sum(x[h,a,w] + x[a,h,w] for h in range(home)) == 1 
                    for a in range(away) for w in range(weeks))

In [17]:
#add constraints that all 12 games that a team played would need to be against a different opponent 
con3 = m1.addConstrs(sum(x[h,a,w] + x[a,h,w] for w in range(weeks)) <= 1
                    for h in range(home) for a in range(away))

In [18]:
#add constraints that each team would play at most 6 home games
con4 = m1.addConstrs(sum(x[h,a,w] for h in range(home) for w in range(weeks)) <= 6 for a in range(away))

In [42]:
#add constraints that no team can plays itself
con = m1.addConstrs((x[h,h,w] == 0 for h in range(home) for w in range(weeks)))

In [19]:
#add constraints that no team would play more than 2 consecutive games in home
con5 = m1.addConstrs(sum(y[h,w] for w in range(weeks)) <= 1 for h in range(home))

In [20]:
#Implication of constraint 5
con5_i = m1.addConstrs(sum(x[h,a,w] + x[h,a,w+1] for a in range(away)) <= 1 + y[h,w] for w in range(weeks-1) for h in range(home))

In [21]:
#add constraints that no team would play more than 2 consecutive games in away
con6 = m1.addConstrs(sum(z[h,w] for w in range(weeks)) <= 1 for h in range(home))

In [28]:
#Implication of constraint 6
con6_i = m1.addConstrs(sum(x[h,a,w] + x[h,a,w+1] for a in range(away)) <= 1 + z[h,w] for w in range(weeks-1) for h in range(home))

In [64]:
#add constraints that schedulers requested the bye week to be assigned on week 9, which means that there are no games at week 9
con7 = m1.addConstrs(sum(x[h,a,w] + x[a,h,w] for h in range(home) for a in range(away) and for w in range(weeks) == 9) == 0)

SyntaxError: invalid syntax (<ipython-input-64-8618497a61a0>, line 2)

#### Optimization

In [25]:
m1.setObjective(obj, GRB.MINIMIZE)
m1.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 2280 rows, 7488 columns and 112776 nonzeros
Model fingerprint: 0x46ef245f
Variable types: 0 continuous, 7488 integer (7488 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 1116 rows and 336 columns
Presolve time: 0.07s
Presolved: 1164 rows, 7152 columns, 51840 nonzeros
Variable types: 0 continuous, 7152 integer (7152 binary)

Root relaxation: objective 1.659080e+05, 5212 iterations, 0.61 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 165908.000    0  432          - 165908.000      -     -    3s
     0     0 165908.000    0  577          - 165908.000      -     -    4s
H    0     0                    172017.99967 165908.000  3.55%     -    4s
H    0     

In [26]:
# ADD PRINTING HERE:
# print the total distance travelled by all teams

if m1.status == GRB.OPTIMAL:
    # Print solution
    print("\nTotal distance: {}\n". format(m1.ObjVal))    

# print the optimal schedule for the decision variables

print('\nDecision variables:')
for v in m1.getVars():
    if v.x== 1:
        s=("%s : %g" % (v.varName, v.obj))
        print(s)


Total distance: 165912.0


Decision variables:
x[0,6,10] : 940
x[0,10,0] : 1292
x[0,13,4] : 922
x[0,15,9] : 1586
x[0,18,6] : 1300
x[0,23,2] : 1586
x[1,0,5] : 490
x[1,8,7] : 1074
x[1,10,3] : 814
x[1,11,8] : 874
x[1,19,1] : 1648
x[1,20,11] : 1250
x[2,3,7] : 568
x[2,4,5] : 414
x[2,5,4] : 818
x[2,12,0] : 1056
x[2,13,2] : 594
x[2,21,10] : 920
x[3,5,10] : 1382
x[3,11,6] : 1056
x[3,12,4] : 540
x[3,13,1] : 526
x[3,14,8] : 340
x[3,16,2] : 574
x[4,3,11] : 978
x[4,5,9] : 558
x[4,12,6] : 1468
x[4,15,2] : 2574
x[4,16,7] : 788
x[4,22,0] : 1366
x[5,0,8] : 2226
x[5,6,11] : 2398
x[5,14,6] : 1504
x[5,15,0] : 2352
x[5,16,3] : 1182
x[5,22,2] : 1758
x[6,1,4] : 1430
x[6,2,9] : 1850
x[6,4,8] : 2260
x[6,18,2] : 1702
x[6,22,6] : 1064
x[6,23,0] : 1010
x[7,1,2] : 1250
x[7,3,9] : 1226
x[7,14,4] : 912
x[7,17,1] : 1868
x[7,19,6] : 408
x[7,21,11] : 728
x[8,7,10] : 192
x[8,9,6] : 2084
x[8,11,11] : 202
x[8,17,8] : 1692
x[8,18,1] : 2358
x[8,20,4] : 192
x[9,0,7] : 912
x[9,1,0] : 1160
x[9,6,3] : 1314
x[9,11,5] : 1882
x[

#### print the total distance travelled by all teams

In [28]:
print('\nValue of objective function: %g' % m1.objVal)


Value of objective function: 165912


In [ ]:
# print the optimal schedule for Cleveland Browns, 14

In [ ]:
# Would you recommend adding the game pattern and bye week constraints?